Import knihoven

In [59]:
import pandas as pd
import scipy


Načtení dat

In [60]:
df = pd.read_pickle("accidents.pkl.gz")
df.head()


,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,n,o,p,q,r,s,t,p5a,region,date
0,2100160001,4,NaN,2016-01-01,5,55,1,1,0,2,...,711403.0,NaN,Souhlasnýsesměremúseku,Pomalý,554782.0,451622.0,GN_V0.1UIR-ADR_410,1,PHA,2016-01-01
1,2100160002,4,NaN,2016-01-01,5,130,1,3,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01
2,2100160003,5,NaN,2016-01-01,5,100,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01
3,2100160004,6,NaN,2016-01-01,5,120,9,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01
4,2100160005,6,NaN,2016-01-01,5,2560,2,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01


# Hypotéza 1
Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na
silnicích třetí třídy.

K ověření hypotézy využijte $\chi^2$ test s tím, že také určíte, jestli nehody na silnicích 1. třídy vedly častěji či méně často k fatální nehodě. V tom vám může pomoct “expected” výstup $\chi^2$ testu. Nerozlišujte počet následků, ale pouze nehody, kde došlo k jednomu a více úmrtí (${p13a} > 0$).

Získání kontingenční tabulky

In [61]:
df["fatal"] = df["p13a"] > 0
df = df[df["p36"].isin([1, 3])]
table = pd.crosstab(df["fatal"], df["p36"])
table


p36,1,3
fatal,,
False,96618,91882
True,1104,536


Chi kvadrát test

In [62]:
test = scipy.stats.chi2_contingency(table)
test


Chi2ContingencyResult(statistic=167.2443757129343, pvalue=2.95835646229767e-38, dof=1, expected_freq=array([[96879.12590723, 91620.87409277],
       [  842.87409277,   797.12590723]]))

Výsledek testu

In [63]:
print("P-value:", test[1])
print("0 hypotesis passed:", test[1] > 0.05)
print("Mean fatal accidents on first class roads:", df[df["p36"] == 1]["fatal"].mean())
print("Mean fatal accidents on third class roads:",
      df[df["p36"] == 3]["fatal"].mean())


P-value: 2.95835646229767e-38
0 hypotesis passed: False
Mean fatal accidents on first class roads: 0.011297353717688955
Mean fatal accidents on third class roads: 0.005799735982167976


## Závěr
P-hodnota je menší než 0.05, takže nulovou hypotézu zamítáme. To znamená, že existuje statisticky významená kolerace mezi třídou silnice a počtem nehod s fatální nehodou. Na silnicích první třídy se při nehodách umíralo s větší pravděpodobností než na silnicích třetí třídy.

# Hypotéza 2
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.

Využijte vhodný test, který vyberete na základě distribuce dat, nevyužívejte $\chi^2$ test. Určete, zda je škoda na vozidlech vyšší či nižší a zda je tato vlastnost na požadované hladině významnosti.

Získání nehod jednotlivých značek

In [64]:
df_skoda = df[df["p45a"] == 39]
df_audi = df[df["p45a"] == 2]
(df_skoda.shape, df_audi.shape)


((47272, 67), (4311, 67))

Otestování, zda mají data normální rozložení

In [65]:
(scipy.stats.normaltest(df_skoda["p53"]), scipy.stats.normaltest(df_audi["p53"]))


(NormaltestResult(statistic=193618.35987947113, pvalue=0.0),
 NormaltestResult(statistic=6115.5561771711555, pvalue=0.0))

Z výsledku je vidět, že data nemají normální rozložení, proto použijeme Mann-Whitneyův test místo T-testu.

In [66]:
test = scipy.stats.mannwhitneyu(df_skoda["p53"], df_audi["p53"])
print("P-value:", test[1])
print("0 hypotesis passed:", test[1] > 0.05)
print("Mean accident costs of Skoda drivers:", df_skoda["p53"].mean())
print("Mean accident costs of Audi drivers:", df_audi["p53"].mean())


P-value: 2.6748732209485882e-64
0 hypotesis passed: False
Mean accident costs of Skoda drivers: 428.9974826535793
Mean accident costs of Audi drivers: 642.2247738343772


## Závěr
Zamítáme hypotézu, že škoda na vozidlech značky Škoda je stejná jako škoda na vozidlech značky Audi. Škoda na vozidlech značky Škoda je nižší než škoda na vozidlech značky Audi.